- This notebook will download the dataset from Chicago Data Portal and put it into S3 bucket.
- The file will then be consumed by SageMaker Data Wrangler that will do transformation and put it into a SageMaker Feature Store.

## 0. Import packages and config

In [ ]:
import os
import time
import yaml
import boto3

bucket=sagemaker.Session().default_bucket()
prefix = "dataset"


## 1. Download file from Chicago Data Portal

In [ ]:
! mkdir dataset

In [ ]:
year = 2023

for month in range(1,13):
    for day in range(1,31):
        file_name = f"dataset/{year}_{month}_{day}.csv"
        year_month_day = f"{year}-{month}-{day}"
        time.sleep(5)
        limit = 1000
        os.system(f"curl -o {file_name} 'https://data.cityofchicago.org/resource/wrvz-psew.csv?$limit={limit}&$offset=&$where=trip_start_timestamp%20between%20%27{year_month_day}T00:00:00%27%20and%20%27{year_month_day}T23:00:00%27'")

## 2. Put the file into S3 bucket

In [ ]:
# instantiate S3 client and upload to s3

files = os.listdir("dataset")
s3 = boto3.resource('s3')

for f in files:
    if ".csv" in f:
        file_path = f"dataset/{f}"
        s3.meta.client.upload_file(file_path, bucket_name, os.path.join(prefix, f))